In [1]:
%%bash
docker run \
  --rm \
  --name omnisci_test \
  --ipc=host \
  -d --runtime=nvidia \
  -v $(readlink -f ..)/data:/data \
  -v $(readlink -f ..)/scripts:/src \
  -p 6273-6280:6273-6280 \
  omnisci/core-os-cuda:v5.2.1

64abb6ded633700a851752ad6e1a045fb35ffda1ba7b505a91e0b86ea315d24c


In [2]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "cat /src/load_dataset.sql | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Result
Loaded: 1067371 recs, Rejected: 0 recs in 0.331000 secs
1 rows returned.
Execution time: 422 ms, Total time: 423 ms
User admin disconnected from database omnisci


In [3]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "echo 'SELECT * FROM online_retail LIMIT 10;' | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Invoice|StockCode|Description|Quantity|InvoiceDate|Price|CustomerID|Country
489612|22086|PAPER CHAIN KIT 50'S CHRISTMAS|6|2009-12-01 14:55:00|6.04|NULL|United Kingdom
489612|22136|LOVE HEART SOCK HANGER|1|2009-12-01 14:55:00|3.43|NULL|United Kingdom
491634|21017|RED BIRD HOUSE TREE DECORATION|2|2009-12-11 15:40:00|0.85|17841.0|United Kingdom
489612|22144|CHRISTMAS CRAFT LITTLE FRIENDS|2|2009-12-01 14:55:00|4.3|NULL|United Kingdom
490741|21872|GLAMOROUS  MUG|1|2009-12-07 17:56:00|3.43|NULL|United Kingdom
491966|22285|DECORATION HEN ON NEST, HANGING|1|2009-12-14 17:16:00|3.43|NULL|United Kingdom
491634|20831|GOLD PHOTO FRAME|1|2009-12-11 15:40:00|2.1|17841.0|United Kingdom
491691|21819|GLITTER CHRISTMAS STAR|10|2009-12-13 13:17:00|0.85|17211.0|United Kingdom
490209|22200|FRYING PAN PINK POLKADOT|72|2009-12-04 12:06:00|4.25|17389.0|United Kingdom
489612|22174|PHOTO CUBE|1|2009-12-01 14:55:00|3.43|NULL|United Kingdom
10 rows returned.
Execution time

In [4]:
from pymapd import connect
from time import time
import pandas as pd
import numpy as np
import json

In [5]:
with open('test_result.json') as json_file:
    test_result = json.load(json_file)

In [6]:
con = connect(user="admin", password="HyperInteractive", host="localhost", dbname="omnisci")

In [7]:
test_result["timings"]["omnisci"] = {}

for label, query in test_result["queries"].items():
    print(f"Executing {label} -> {query}")
    test_result["timings"]["omnisci"][label] = []
    for i in range(0, 30):
        start = time()
        pd.read_sql_query(query, con=con)  # <-un po' più veloce ma usa ipc (arrow)
        # df = con.select_ipc_gpu(query)  # <-analogo
        # result = list(con.execute(query))  #NO IPC
        end = time()

        total = end-start
        test_result["timings"]["omnisci"][label].append(total)

with open('test_result.json', 'w') as outfile:
    json.dump(test_result, outfile, indent=2)

Executing Q1 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail GROUP BY Invoice
Executing Q2 -> SELECT CustomerID, SUM(Price*Quantity) as customer_sum FROM online_retail GROUP BY CustomerID
Executing Q3 -> SELECT Country, SUM(Price*Quantity) as country_sum FROM online_retail GROUP BY Country
Executing Q4 -> SELECT StockCode, COUNT(*) as n_items FROM online_retail GROUP BY StockCode ORDER BY n_items DESC
Executing Q5 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail GROUP BY Invoice ORDER BY invoice_sum DESC
Executing Q6 -> SELECT InvoiceDate, AVG(Price*Quantity) as avg_total FROM online_retail GROUP BY InvoiceDate ORDER BY avg_total
Executing Q7 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail WHERE Country='France' GROUP BY Invoice ORDER BY invoice_sum DESC
Executing Q8 -> SELECT CustomerID, Price*Quantity as total_price FROM online_retail WHERE Quantity > 20 ORDER BY total_price DESC
Executing Q9 -> SELECT Customer

## Join tests

In [10]:
# query = "SELECT Invoice, stock_codes.Price FROM online_retail JOIN stock_codes ON online_retail.StockCode = stock_codes.StockCode"

# start = time()
# pd.read_sql_query(query, con=con)
# end = time()

# total = end-start
# print(total)

1.6789634227752686


In [11]:
# query = "SELECT Invoice, Price FROM online_retail"

# start = time()
# pd.read_sql_query(query, con=con)
# end = time()

# total = end-start
# print(total)

1.4988090991973877


In [8]:
%%bash
docker stop omnisci_test

omnisci_test
